In [1]:
import os
import google.generativeai as genai
from dotenv import load_dotenv
load_dotenv()
import json
import numpy as np

# Initialize embeddings model
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

# Cache for knowledge base data and embeddings
_knowledge_base_cache = None
_embeddings_cache = None

/Users/brauliopf/Documents/Dev/Cloud/swarm-google-adk/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
from pathlib import Path
import json

# Define the path
DB_FILE = Path("../data/mock_kb.json")

# Load the database
def load_db():
    if not DB_FILE.exists():
        return {}
    with open(DB_FILE, "r") as f:
        return json.load(f)

load_db()

{'knowledge_base': [{'url': 'https://www.infinitepay.io/maquininha-celular',
   'title': 'Maquininha Celular',
   'content': 'O InfiniteTap da InfinitePay transforma seu celular em uma maquininha de cartão, permitindo aceitar pagamentos por aproximação com taxas competitivas. As taxas diminuem conforme o faturamento, e o recebimento pode ser imediato ou em 1 dia útil.\n\nA gestão é facilitada com uma conta digital que oferece Pix ilimitado e cartões com cashback. É possível aceitar as principais bandeiras e carteiras digitais. Para usar, basta abrir o app InfinitePay, selecionar "InfiniteTap" ou "Tap to Pay no iPhone", digitar o valor, clicar em "Cobrar" e pedir para o cliente aproximar o cartão, celular ou smartwatch.\n\nRecursos adicionais incluem catálogo de produtos, Pix GRÁTIS, controle de finanças, link de pagamento, empréstimo facilitado e loja online. As taxas são as mesmas da Maquininha Smart da InfinitePay, a partir de 0,75%.\n\nO InfiniteTap funciona em Android 10 com NFC ou

In [20]:
def load_knowledge_base() -> list[dict]:
    """Load knowledge base data from mock.json file."""
    global _knowledge_base_cache

    if _knowledge_base_cache is not None:
        return _knowledge_base_cache

    notebook_dir = os.getcwd()
    parent_dir = os.path.dirname(notebook_dir)

    mock_data_path = os.path.join(
        parent_dir,
        "data",
        "mock.json"
    )

    with open(mock_data_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    _knowledge_base_cache = data.get("knowledge_base", [])
    return _knowledge_base_cache

load_knowledge_base()

[{'url': 'https://www.infinitepay.io/maquininha-celular',
  'title': 'Maquininha Celular',
  'content': 'O InfiniteTap da InfinitePay transforma seu celular em uma maquininha de cartão, permitindo aceitar pagamentos por aproximação com taxas competitivas. As taxas diminuem conforme o faturamento, e o recebimento pode ser imediato ou em 1 dia útil.\n\nA gestão é facilitada com uma conta digital que oferece Pix ilimitado e cartões com cashback. É possível aceitar as principais bandeiras e carteiras digitais. Para usar, basta abrir o app InfinitePay, selecionar "InfiniteTap" ou "Tap to Pay no iPhone", digitar o valor, clicar em "Cobrar" e pedir para o cliente aproximar o cartão, celular ou smartwatch.\n\nRecursos adicionais incluem catálogo de produtos, Pix GRÁTIS, controle de finanças, link de pagamento, empréstimo facilitado e loja online. As taxas são as mesmas da Maquininha Smart da InfinitePay, a partir de 0,75%.\n\nO InfiniteTap funciona em Android 10 com NFC ou iPhone XS ou superio

In [16]:
def get_embedding(text: str, model: str = "models/text-embedding-004") -> np.ndarray:
    """Generate embedding for given text using Google's embedding model."""
    result = genai.embed_content(
        model=model,
        content=text,
        task_type="retrieval_document"
    )
    return np.array(result['embedding'])


def compute_embeddings(knowledge_base: list[dict]) -> list[np.ndarray]:
    """Compute embeddings for all content in the knowledge base."""
    global _embeddings_cache

    if _embeddings_cache is not None:
        return _embeddings_cache

    embeddings = []
    for item in knowledge_base:
        content = item.get("content", "")
        title = item.get("title", "")

        # Combine title and content for better context
        text_to_embed = f"{title}\n\n{content}"
        embedding = get_embedding(text_to_embed)
        embeddings.append(embedding)

    _embeddings_cache = embeddings
    return embeddings


def cosine_similarity(vec1: np.ndarray, vec2: np.ndarray) -> float:
    """Calculate cosine similarity between two vectors."""
    dot_product = np.dot(vec1, vec2)
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    return dot_product / (norm1 * norm2)


def query_knowledge_base(query: str, top_k: int = 2) -> str:
    """
    Query the knowledge base using semantic search with embeddings.

    Args:
        query: The user's search query
        top_k: Number of top results to return (default: 2)

    Returns:
        A formatted string containing the most relevant results
    """
    # Load knowledge base and compute embeddings
    knowledge_base = load_knowledge_base()
    kb_embeddings = compute_embeddings(knowledge_base)

    # Get query embedding
    query_embedding = get_embedding(query, model="models/text-embedding-004")

    # Calculate similarities
    similarities = []
    for i, kb_embedding in enumerate(kb_embeddings):
        similarity = cosine_similarity(query_embedding, kb_embedding)
        similarities.append((i, similarity))

    # Sort by similarity (descending)
    similarities.sort(key=lambda x: x[1], reverse=True)

    # Get top-k results
    top_results = similarities[:top_k]

    # Format results
    results = []
    for idx, score in top_results:
        item = knowledge_base[idx]
        results.append(f"""
**{item['title']}** (Relevance: {score:.2f})
URL: {item['url']}

{item['content']}
""")

    return "\n---\n".join(results)

In [17]:
ans = query_knowledge_base("A InfinitePay anuncia meus produtos?")

In [18]:

print(ans)


**Loja Online** (Relevance: 0.76)
URL: https://www.infinitepay.io/loja-online

A InfinitePay oferece um serviço gratuito de criação de loja online, onde você não precisa de um site, pois eles criam um link para sua loja. Os clientes podem comprar facilmente e os fundos são depositados rapidamente (no mesmo dia ou em 1 dia útil). É possível oferecer parcelamento em até 12 vezes com taxas competitivas e receber o valor total da venda quando os clientes pagam via Pix. Você pode listar produtos ilimitados com fotos, nomes e descrições, e os clientes podem visualizar seu catálogo de produtos e comprar rapidamente via cartão de crédito ou Pix. Acompanhe vendas, informações de clientes e detalhes de pagamento, com gerenciamento automático de estoque disponível. Integre com Melhor Envio para cálculo automático de frete ou ofereça opções de entrega combinadas. Para criar sua loja online, baixe o aplicativo InfinitePay e crie uma conta, clique em "Sua Loja Online" e depois em "Novo Produto", pr